### Assignment2

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models

In [2]:
#load cifar10
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

#train validation set
trainval = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
#split trainval into trainset and valset 
trainset,valset = torch.utils.data.random_split(trainval, [0.9, 0.1])
#trainloader and valloader
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=0) 
valloader = torch.utils.data.DataLoader(valset, batch_size=4, shuffle=True, num_workers=0)
#test set
testset = trainval = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
#testloader
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=0)
#classes
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

100%|████████████████████████████████████████████████████████████████| 170498071/170498071 [11:22<00:00, 249879.34it/s]


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [4]:
#cuda device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [7]:
#cnn model
model_vgg16 = models.vgg16(weights=None)  
model_vgg16.classifier[6] = nn.Linear(4096, 10) 
model_vgg16 = model_vgg16.to(device)  
print(model_vgg16)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_vgg16.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

In [12]:
num_epochs = 20
for epoch in range(num_epochs):  # 
    model_vgg16.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model_vgg16(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(trainloader)}")

Epoch [1/20], Loss: 1.8791778131642125
Epoch [2/20], Loss: 1.3622799191454595
Epoch [3/20], Loss: 1.0508810736408287
Epoch [4/20], Loss: 0.860659631731158
Epoch [5/20], Loss: 0.7233399897813797
Epoch [6/20], Loss: 0.6215460762296888
Epoch [7/20], Loss: 0.5093617305074903
Epoch [8/20], Loss: 0.4376234319272705
Epoch [9/20], Loss: 0.37269407337192784
Epoch [10/20], Loss: 0.3158816680343906
Epoch [11/20], Loss: 0.2775711109120907
Epoch [12/20], Loss: 0.251955400205175
Epoch [13/20], Loss: 0.20118494518630375
Epoch [14/20], Loss: 0.17923740912837358
Epoch [15/20], Loss: 0.16907915693379685
Epoch [16/20], Loss: 0.1466956781951012
Epoch [17/20], Loss: 0.13864712508802768
Epoch [18/20], Loss: 0.12209803899366473
Epoch [19/20], Loss: 0.10548878362028931
Epoch [20/20], Loss: 0.10093225305436962


In [15]:
model_vgg16.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model_vgg16(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy: {100 * correct / total:.2f}%")

Accuracy: 80.72%
